In [1]:
import numpy as np
import cv2
from keras.models import load_model

In [2]:
model = load_model('model.h5')

In [3]:
face_cascade = cv2.CascadeClassifier('cascade/haarcascade_frontalface_default.xml')

In [32]:
capture = cv2.VideoCapture(0)

while True :
    ret, image = capture.read()

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
     
    for (x, y, w, h) in faces:
        def transparentOverlay(src, overlay, pos=(0,0)):
            h, w, _ = overlay.shape  # Size of foreground
            rows, cols, _ = src.shape  # Size of background Image
            y, x = pos[0], pos[1]    # Position of foreground/overlay image
            
            #loop over all pixels and apply the blending equation
            for i in range(h):
                for j in range(w):
                    if x+i >= rows or y+j >= cols:
                        continue
                    alpha = float(overlay[i][j][3]/255.0) # read the alpha channel 
                    src[x+i][y+j] = alpha * overlay[i][j][:3] + (1-alpha) * src[x+i][y+j]
            return src


        # cv2.rectangle(image, (x,y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]
        
        width_original = roi_gray.shape[1] 
        height_original = roi_gray.shape[0]

        img_gray = cv2.resize(roi_gray, (96, 96))
        img_gray = img_gray/255
        
        img_model = np.reshape(img_gray, (1,96,96,1))
        keypoints = model.predict(img_model)[0]
        
        x_coords = keypoints[0::2]
        y_coords = keypoints[1::2]

        x_coords_denormalized = (x_coords / 96) * width_original + x 
        y_coords_denormalized = (y_coords / 96) * height_original + y

        # for i in range(15) :
        #     cv2.circle(image, (round(x_coords_denormalized[i]), round(y_coords_denormalized[i])), 2, (255, 255, 0), -1)

        ## Coordinates for each facial keypoint

        left_eye_center = (round(x_coords_denormalized[0]), round(y_coords_denormalized[0]))
        right_eye_center = (round(x_coords_denormalized[1]), round(y_coords_denormalized[1]))
        left_eye_inner_corner = (round(x_coords_denormalized[2]), round(y_coords_denormalized[2]))
        left_eye_outer_corner = (round(x_coords_denormalized[3]), round(y_coords_denormalized[3]))
        right_eye_inner_corner = (round(x_coords_denormalized[4]), round(y_coords_denormalized[4]))
        right_eye_outer_corner = (round(x_coords_denormalized[5]), round(y_coords_denormalized[5]))
        left_eyebrows_inner_end = (round(x_coords_denormalized[6]), round(y_coords_denormalized[6]))
        left_eyebrows_outer_end = (round(x_coords_denormalized[7]), round(y_coords_denormalized[7]))
        right_eyebrows_inner_end = (round(x_coords_denormalized[8]), round(y_coords_denormalized[8]))
        right_eyebrows_outer_end = (round(x_coords_denormalized[9]), round(y_coords_denormalized[9]))
        nose_tip = (round(x_coords_denormalized[10]), round(y_coords_denormalized[10]))
        mouth_left_corner = (round(x_coords_denormalized[11]), round(y_coords_denormalized[11]))
        mouth_right_corner = (round(x_coords_denormalized[12]), round(y_coords_denormalized[12]))
        mouth_center_top_lip = (round(x_coords_denormalized[13]), round(y_coords_denormalized[13]))
        mouth_center_bottom_lip = (round(x_coords_denormalized[14]), round(y_coords_denormalized[14]))

        ## Sunglasses Filter

        # glasses_width =  max(1, abs(left_eye_outer_corner[0] - right_eye_outer_corner[0]))
        # glasses = cv2.imread('images\sunglasses.png', -1)
        # glasses = cv2.resize(glasses, (round(glasses_width * 1.6), round(h/3)))
        # image = transparentOverlay(image, glasses, (right_eyebrows_outer_end[0] - 10, right_eyebrows_outer_end[1] - 10))
        
        ## Pig Nose Filter

        # nose = cv2.imread('images\pig_nose.png', -1)
        # nose = cv2.resize(nose, (int(w/5), int(h/5)))
        # image = transparentOverlay(image, nose, (nose_tip[0] - int(w/10), nose_tip[1] - int(h/10)))
        
        ## Cat Whisker Filter

        # whisker = cv2.imread('images\cat_whisker.png', -1)
        # whisker = cv2.resize(whisker, (int(w), int(h/2)))
        # image = transparentOverlay(image, whisker, (nose_tip[0] - int(w/2), nose_tip[1] - int(h/4)))

        ## Dog Filter

        # dog_filter = cv2.imread('images\dog_filter_1.png', -1)
        # dog_filter = cv2.resize(dog_filter, (int(w*1.5), int(h*1.25*1.5)))
        # image = transparentOverlay(image, dog_filter, (nose_tip[0] - round(w*0.8), nose_tip[1] - round(h*1.2)))

        ## Graduation Cap Filter

        # grad_cap = cv2.imread('images\graduation_cap.png', -1)
        # grad_cap = cv2.resize(grad_cap, (w*2, h))
        # image = transparentOverlay(image, grad_cap, (right_eyebrows_outer_end[0] - w + round(w/3), right_eyebrows_outer_end[1] - h))

        ## Mustache Filter

        # mustache_width =  max(1, abs(mouth_left_corner[0] - mouth_right_corner[0]))
        # mustache = cv2.imread('images\mustache.png', -1)
        # mustache = cv2.resize(mustache, (round(mustache_width*2), round(h/1.25)))
        # mustache = transparentOverlay(image, mustache, (mouth_center_top_lip[0] - round(w/3), mouth_center_top_lip[1] - round(h/2.4)))

        ## Clown Wig Filter

        # clown_wig = cv2.imread('images\clown_wig.png', -1)
        # clown_wig = cv2.resize(clown_wig, (round(w*1.6), round(h*1.2)))
        # image = transparentOverlay(image, clown_wig, (right_eyebrows_outer_end[0] - round(w*0.55), right_eyebrows_outer_end[1] - h))

    cv2.imshow('Webcam',image)

    if cv2.waitKey(1) & 0xFF == ord("q") :
        break

capture.release()
cv2.destroyAllWindows()
